In [ ]:
# Uninstall problematic pre-installed versions
!pip uninstall -y tensorflow tensorflow-text tensorflow-decision-forests tf-keras

# Install required compatible versions
!pip install -q torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0
!pip install -q onnx
!pip install -q efficientnet_pytorch



Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: tensorflow-text 2.18.1
Uninstalling tensorflow-text-2.18.1:
  Successfully uninstalled tensorflow-text-2.18.1
Found existing installation: tensorflow_decision_forests 1.11.0
Uninstalling tensorflow_decision_forests-1.11.0:
  Successfully uninstalled tensorflow_decision_forests-1.11.0
Found existing installation: tf_keras 2.18.0
Uninstalling tf_keras-2.18.0:
  Successfully uninstalled tf_keras-2.18.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import torch
from efficientnet_pytorch import EfficientNet
import timm
import torchvision.models as models


# Load torchvision model
model = models.efficientnet_b0(pretrained=False)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, 28)

# Load trained weights
state_dict = torch.load('/content/drive/MyDrive/efficientnet_rfmid_best.pth', map_location=torch.device('cpu'))
model.load_state_dict(state_dict)
model.eval()

# Create an example entry (1, 3, 224, 224)
dummy_input = torch.randn(1, 3, 224, 224)

# Export to ONNX
onnx_path = '/content/efficientnet_rfmid_best.onnx'
torch.onnx.export(model, dummy_input, onnx_path,
                  input_names=['input'], output_names=['output'],
                  dynamic_axes={'input': {0: 'batch'}, 'output': {0: 'batch'}},
                  opset_version=11)
print("ONNX model stored in:", onnx_path)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Modelo ONNX guardado en: /content/efficientnet_rfmid_best.onnx


In [ ]:
from google.colab import drive
import shutil

# Montar Google Drive
drive.mount('/content/drive')

# Ruta del archivo ONNX generado
onnx_file_path = '/content/efficientnet_rfmid_best.onnx'

# Ruta de destino en tu Google Drive (puedes modificarla)
destination_path = '/content/drive/MyDrive/efficientnet_rfmid_best.onnx'

# Copiar archivo al destino
shutil.copy(onnx_file_path, destination_path)

print(f"File copied to: {destination_path}")

# The following part was done locally with the convert_onnx_to_tflite script due to problems with dependencies in colab.
